# 인간 피드백으로 얻은 항목 추가

![Pipeline](img/generative_ai_pipeline_rlhf_plus.png)

![RLHF](img/rlhf_qa.png)

In [7]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=33163763712, available=31108259840, percent=6.2, used=1630367744, free=28735410176, active=525201408, inactive=3470077952, buffers=4329472, cached=2793656320, shared=638976, slab=181776384)


In [8]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [9]:
import io
import json
import uuid
import time
import boto3
import botocore

# 아마존 파이썬 SDK 클라이언트
sagemaker = boto3.client("sagemaker", region)
#comprehend = boto3.client("comprehend", region)
a2i = boto3.client("sagemaker-a2i-runtime")
s3 = boto3.client("s3", region)

# 이전에 생성된 `augmented_ai_flow_definition_arn` 가져오기

In [10]:
%store -r augmented_ai_flow_definition_arn

In [11]:
print(augmented_ai_flow_definition_arn)

arn:aws:sagemaker:us-east-1:843956416240:flow-definition/fd-ranking-1aa49519-cb36-4b67-abe7-983f5c40d5d8


In [12]:
items = [
    {
        "prompt": """
            Chris: Hey Antje and Shelbee! Do you want to write a book on generative AI?
            Antje: That sounds fun. What do you think, Shelbee?
            Shelbee: Of course! Should we title the book, “Generative AI on AWS”?
            Chris: Yes!
            Antje: Yes!
        """,
        "responses": [
            """
            Chris, Shelbee, and Antje agree to write a book titled, “Generative AI on AWS”.
            """,
            """
            Chris asks Antje and Shelbee if they want to write a book on generative AI. They agree to write the book.
            """,
            """
            Chris asks Antje and Shelbee to write a book. They agree.
            """,
            """
            Chris, Antje, and Shelbee decide not to write a book.
            """
        ]
    }
]

In [13]:
human_loops_started = []

for item in items:
    print(f'Processing item: "{item}"')

    humanLoopName = str(uuid.uuid4())
    inputContent = {"taskObject": item}
    start_loop_response = a2i.start_human_loop(
        HumanLoopName=humanLoopName,
        FlowDefinitionArn=augmented_ai_flow_definition_arn,
        HumanLoopInput={"InputContent": json.dumps(inputContent)},
    )

    human_loops_started.append(humanLoopName)

    print(f"*** ==> Starting human loop with name: {humanLoopName}  \n")

Processing item: "{'prompt': '\n            Chris: Hey Antje and Shelbee! Do you want to write a book on generative AI?\n            Antje: That sounds fun. What do you think, Shelbee?\n            Shelbee: Of course! Should we title the book, “Generative AI on AWS”?\n            Chris: Yes!\n            Antje: Yes!\n        ', 'responses': ['\n            Chris, Shelbee, and Antje agree to write a book titled, “Generative AI on AWS”.\n            ', '\n            Chris asks Antje and Shelbee if they want to write a book on generative AI. They agree to write the book.\n            ', '\n            Chris asks Antje and Shelbee to write a book. They agree.\n            ', '\n            Chris, Antje, and Shelbee decide not to write a book.\n            ']}"
*** ==> Starting human loop with name: 75fd992e-867c-4741-99f2-ace6ffa50435  



In [14]:
%store human_loops_started

Stored 'human_loops_started' (list)


# 휴먼 루프 상태 확인하기

In [15]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")

    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

HumanLoop Name: 75fd992e-867c-4741-99f2-ace6ffa50435
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-843956416240/ground-truth-star-rating-results/fd-ranking-1aa49519-cb36-4b67-abe7-983f5c40d5d8/2024/08/25/01/53/19/75fd992e-867c-4741-99f2-ace6ffa50435/output.json'}



# 휴먼 루프 작업이 완료될 때까지 기다리기

아래 링크로 이동하여, 개인 워크포스(Private Workforce) 설정 시 사용한 이메일과 비밀번호로 로그인하세요.

In [16]:
%store -r augmented_ai_workteam_arn

In [17]:
print(augmented_ai_workteam_arn)

arn:aws:sagemaker:us-east-1:843956416240:workteam/private-crowd/dsoaws


In [18]:
workteam_name = augmented_ai_workteam_arn[augmented_ai_workteam_arn.rfind("/") + 1 :]
print(workteam_name)
print("Navigate to the private worker portal and complete the human loop.")
print("Make sure you have invited yourself to the workteam and received the signup email.")
print("Note:  Check your spam filter if you have not received the email.")
print("")
print("https://" + sagemaker.describe_workteam(WorkteamName=workteam_name)["Workteam"]["SubDomain"])

dsoaws
Navigate to the private worker portal and complete the human loop.
Make sure you have invited yourself to the workteam and received the signup email.
Note:  Check your spam filter if you have not received the email.

https://d5hmgwn4tg.labeling.us-east-1.sagemaker.aws


# 진행하기 전에 위의 링크를 클릭하여 데이터를 라벨링해야 합니다!!

# 라벨링 시작하기

<img src="img/augmented-comprehend-custom-start-working.png" width="80%" align="left">

# 레이블 선택하기

<img src="img/augmented-comprehend-custom-select-label.png" width="80%" align="left">

# 루프 완료

<img src="img/augmented-comprehend-custom-finished-task.png" width="80%" align="left">

# 휴먼 루프 완료 여부 확인하기

In [19]:
workteam_name = augmented_ai_workteam_arn[augmented_ai_workteam_arn.rfind("/") + 1 :]
print(workteam_name)
print("Navigate to the private worker portal and complete the human loop.")
print("Make sure you have invited yourself to the workteam and received the signup email.")
print("Note:  Check your spam filter if you have not received the email.")
print("")
print("https://" + sagemaker.describe_workteam(WorkteamName=workteam_name)["Workteam"]["SubDomain"])

dsoaws
Navigate to the private worker portal and complete the human loop.
Make sure you have invited yourself to the workteam and received the signup email.
Note:  Check your spam filter if you have not received the email.

https://d5hmgwn4tg.labeling.us-east-1.sagemaker.aws


In [21]:
import time

completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")
    while resp["HumanLoopStatus"] != "Completed":
        print(f"Waiting for HumanLoop to complete.")
        time.sleep(10)
        resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)
        print(f"Completed!")
        print("")

HumanLoop Name: 75fd992e-867c-4741-99f2-ace6ffa50435
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-843956416240/ground-truth-star-rating-results/fd-ranking-1aa49519-cb36-4b67-abe7-983f5c40d5d8/2024/08/25/01/53/19/75fd992e-867c-4741-99f2-ace6ffa50435/output.json'}

Completed!



# 안간 레이블 확인하기

작업이 완료되면, 아마존 그라운드 트루스는 결과를 지정된 S3 버킷에 저장하고 CloudWatch 이벤트를 보냅니다. 아래는 그라운드 트루스에서 jsonlines 형식으로 레이블이 매겨진 샘플 항목입니다.


```
{
 "inputContent": {"taskObject": {
                         "prompt": "Who is Angela Merkel's favorite President of the United States?",
                         "responses": ["George Clinton", "Barack Obama"]}
                 },
 "humanAnswers": [{"answerContent": {
                        "ranking_1": "1", # 첫번째 응답 순위 (1=최우수)
                        "ranking_2": "2"  # 두번째 응답 순위 (2=최악)
                 }}]
}
```

# 강화 학습/PPO 학습을 위한 인간이 단 주석 데이터 준비하기
그라운드 트루스에서 데이터를 가져와 모든 순위에 대해 이진 보상(-1, 1)으로 변환합니다.




이진 보상 변환 전:
```
prompt                                                              response           ranking

Who is Angela Merkel's favorite President of the United States?     George Clinton     1   # 최우수
Who is Angela Merkel's favorite President of the United States?     Barack Obama       2   # 최악
```

이진 보상 변환 후:
```
prompt                                                              response           reward

Who is Angela Merkel's favorite President of the United States?     George Clinton     0   # 낮은 보상
Who is Angela Merkel's favorite President of the United States?     Barack Obama       1   # 높은 보상
```

이진 보상 변환 후:
```
prompt                                                              response                               highest_ranked_response

Who is Angela Merkel's favorite President of the United States?     ["George Clinton", "Barack Obama"]     [0, 1]
```

# 참고: 아래에 아무것도 표시되지 않으면, 이전 노트북에서 그라운드 트루스에 대한 데이터 라벨링하는 작업을 완료해야 합니다!!

In [22]:
import re
from pprint import pprint

human_feedback_items = []

for resp in completed_human_loops:
    human_feedback_s3_uri = resp["HumanLoopOutput"]["OutputS3Uri"]
    split_string = re.split("s3://" + bucket + "/", resp["HumanLoopOutput"]["OutputS3Uri"])
    key = split_string[1]

    response = s3.get_object(Bucket=bucket, Key=key)
    content = response["Body"].read().decode("utf-8")
    json_output = json.loads(content)

    prompt = json_output["inputContent"]['taskObject']['prompt']
    responses = json_output["inputContent"]['taskObject']['responses']
    response_1_ranking = json_output["humanAnswers"][0]["answerContent"]['response_1_ranking']
    response_2_ranking = json_output["humanAnswers"][0]["answerContent"]['response_2_ranking']

    human_feedback_item_1 = (prompt, responses[0], response_1_ranking)
    human_feedback_items.append(human_feedback_item_1)
    human_feedback_item_2 = (prompt, responses[1], response_2_ranking)
    human_feedback_items.append(human_feedback_item_2)

In [23]:
df_human_feedback_items = pd.DataFrame(human_feedback_items, columns=['prompt', 'response', 'ranking'])
df_human_feedback_items.head(10)

,prompt,response,ranking
0,\n Chris: Hey Antje and Shelbee! Do...,"\n Chris, Shelbee, and Antje agree ...",2
1,\n Chris: Hey Antje and Shelbee! Do...,\n Chris asks Antje and Shelbee if ...,1


# 순위를 0 또는 1 보상으로 변환

In [24]:
num_rankings = 2
df_human_feedback_items['response'] = df_human_feedback_items['response'].apply(lambda response: str(response))
df_human_feedback_items['ranking'] = df_human_feedback_items['ranking'].apply(lambda ranking: str(abs(int(ranking) - num_rankings)))
df_human_feedback_items.head(10)

,prompt,response,ranking
0,\n Chris: Hey Antje and Shelbee! Do...,"\n Chris, Shelbee, and Antje agree ...",0
1,\n Chris: Hey Antje and Shelbee! Do...,\n Chris asks Antje and Shelbee if ...,1


In [25]:
df_human_feedback_items_grouped_by_prompt = df_human_feedback_items.groupby('prompt', as_index=False).agg({'prompt' : 'first', 'response' : ','.join, 'ranking' : ','.join})
df_human_feedback_items_grouped_by_prompt

,prompt,response,ranking
0,\n Chris: Hey Antje and Shelbee! Do...,"\n Chris, Shelbee, and Antje agree ...","0,1"


In [26]:
df_human_feedback_items_grouped_by_prompt['response'] = df_human_feedback_items_grouped_by_prompt['response'].apply(lambda response: [s for s in response.split(',')])
df_human_feedback_items_grouped_by_prompt['ranking'] = df_human_feedback_items_grouped_by_prompt['ranking'].apply(lambda ranking: [int(s) for s in ranking.split(',')])
df_human_feedback_items_grouped_by_prompt

,prompt,response,ranking
0,\n Chris: Hey Antje and Shelbee! Do...,"[\n Chris, Shelbee, and Antje agr...","[0, 1]"


In [27]:
from datasets import Dataset

# Create Dataset objects (Arrow PyTables) from Pandas dataframes
human_feedback_dataset = Dataset.from_pandas(df_human_feedback_items_grouped_by_prompt)
human_feedback_dataset

Dataset({
    features: ['prompt', 'response', 'ranking'],
    num_rows: 1
})

In [28]:
%store human_feedback_dataset

Stored 'human_feedback_dataset' (Dataset)


# 계속하기 전에 위의 링크를 클릭하여 데이터를 라벨링해야 합니다!!

In [29]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}
</script>